In [1]:
import collections

from decimal import Decimal
from decimal import getcontext
import math
import itertools
from functools import reduce

import bisect
import string

from baseconv import BaseConverter
import string
import sys

In [2]:
def log2(f):
    return Decimal(math.log2(f))

def entropy(probs):
    return -sum(p * log2(p) for p in probs)

In [3]:
ctx = getcontext()
ctx.prec = 100

In [4]:
probs = {
    'a': Decimal('.1'),
    'b': Decimal('.4'), 
    'c': Decimal('.3'),
    'd': Decimal('.2')
}

In [5]:
class Tanstell:
    def __init__(self, probs, n, d=2):
        assert sum(probs.values()) == 1
        assert 2 <= d <= 16
        
        self.probs = probs
        self.D = d
        self.base = BaseConverter(string.hexdigits[:d])
        self.n = n
        
        L = len(probs)
        q = (d**n - L) // (L - 1)

        self.L = L
        self.q = q

        tanstell_set = collections.Counter({'': Decimal(1.)})
        for i in range(q or 1):
            k, v = tanstell_set.most_common(1)[0]
            tanstell_set.pop(k)

            for s, p in probs.items():
                tanstell_set[k + s] = v * p
        
        codes = {}
        icodes = {}
        for i, k in enumerate(tanstell_set.keys()):
            enc = self.base.encode(i)
            enc = self.base.digits[0] * (n - len(enc)) + enc
            codes[k] = enc
            icodes[enc] = k
        self.codes = codes 
        self.icodes = icodes
    
    def decode(self, seq):
        assert not len(seq) % self.n
        return ''.join([self.icodes[seq[i*self.n:(i+1)*self.n]] for i in range(len(seq) // self.n)])
    
    def encode(self, seq):
        result = ''
        for j in range(len(seq)):
            for i in range(1, len(seq)+1):
                if seq[:i] in self.codes:
                    result += self.codes[seq[:i]]
                    seq = seq[i:]
            
        return result
        
    def prob(self, seq):
        return reduce(lambda x, y: x * y, [self.probs[c] for c in seq])
    
    @property
    def K(self):
        return sum(self.prob(word) * len(word) for word in self.codes.keys())
    
    @property
    def W(self):
        return self.n / self.K
    
    def entropy(self):
        return -sum(self.prob(w) * Decimal(math.log(self.prob(w), self.D)) for w in self.codes.keys())
        
    def entropyU(self):
        return -sum(p * Decimal(math.log(p, self.D)) for w, p in self.probs.items())

In [6]:
t = Tanstell(probs, 5, 7)

print(t.K, t.W)

assert t.W > t.entropyU()

7.4914331320 0.6674290368610874760992260299067166525167350315741962628786900049714012450989063970730774187466911505


In [7]:
lengths = []
sym_probs = []

for n, d in [(4, 2), (4, 3), (4, 2), (5, 5)]:
    t = Tanstell(probs, n, d)
    sys.stdout.write("\r{} {}".format(n, d))
    for i in itertools.product(probs.keys(), repeat=8):
        symbol = ''.join(i)
        encoded = t.encode(symbol)
        lengths.append(len(encoded))
        sym_probs.append(t.prob(symbol))
        # 
        assert t.decode(encoded) in symbol, "{}, {}, {}".format(symbol, t.decode(encoded), encoded)


5 5

In [8]:
class Geometric:
    def __init__(self, probs, d=2):
        assert sum(probs.values()) == 1
        self.probs = probs

        self.offsets = {}
        self.symbols = []
        self.__offs = []
        
        curr = Decimal(0)
        for c, p in probs.items():
            self.offsets[c] = curr
            
            self.symbols.append(c)
            self.__offs.append(curr)
            
            curr += p
                
    def get_interval(self, seq):
        left = Decimal(0)
        width = Decimal(1)
        
        for c in seq:
            left += self.offsets[c] * width
            width *= self.probs[c]
        
        return left, left+width, width
    
    def prob(self, seq):
        return reduce(lambda x, y: x * y, [self.probs[c] for c in seq])
    
    def encode(self, seq):
        left, right, width = self.get_interval(seq)
        
        n = math.ceil(log2(1 / width))
    
        pointer = Decimal(0)
        res = ''

        for i in range(1, n+1):
            if pointer + Decimal(2) ** -i < right:
                pointer += Decimal(2) ** -i
                res += '1'
            else:
                res += '0'

        return res
    
    def decode(self, encoded):
        width = Decimal(2) ** -len(encoded)

        left = Decimal(int(encoded, 2)) / Decimal(2 ** len(encoded))
        right = left + width

        res = ''
        offset = left
        # why there 2 * w?
        while 2 * width < 1:
            ind = bisect.bisect_right(self.__offs, offset) - 1
            sym = self.symbols[ind]
            offset = (offset - self.offsets[sym]) / self.probs[sym]
            width /= self.probs[sym]
            res += sym
        
        return res

In [9]:
t = Geometric(probs)
alphabet = 'abcd'

for length in [1, 2, 5, 10, 15]:
    for line in itertools.combinations_with_replacement(alphabet, length):
        line = ''.join(line)
        l, r, w = t.get_interval(line)

        enc = t.encode(line)
        val = Decimal(int(enc, 2)) / Decimal(2 ** len(enc))
        assert l <= val < r, "{} <= {} < {}".format(l, val, r)
        assert line == t.decode(enc), "{} -> {}".format(line, t.decode(enc))

In [10]:
probs = {
    'a': Decimal('.1'),
    'b': Decimal('.4'), 
    'c': Decimal('.25'),
    'd': Decimal('.15'),
    'e': Decimal('.1')
}

# probs = {
#     'a': D('.03'),
#     'c': D('.03'),
#     'd': D('.03'),
#     'e': D('.91')
# }
assert sum(probs.values()) == 1, sum(probs.values())

In [11]:
t = Geometric(probs)
n = 7

lengths = []
sym_probs = []
for i in itertools.product(probs.keys(), repeat=n):
    symbol = ''.join(i)
    lengths.append(len(t.encode(symbol)))
    sym_probs.append(t.prob(symbol))
